In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [3]:
url = "https://www.detik.com"
response = requests.get(url)
conn = sqlite3.connect('scrap.news.db')

In [4]:
soup = BeautifulSoup(response.text, "lxml")
links = soup.find_all("a")
data = []

In [5]:
for link in links:
    title = ""
    data_label = link.get("data-label")
    data_text = link.text.replace("\n","")
    data_href = link.get("href")

    if link.find("img") != None:
        title = link.find("img").get("title")

    # if link.find("h3") != None:
    #     title = link.find("h3").text

    if title == None or title == "":
        title = data_text

    row = [data_label,title,data_href]
    data.append(row)
    response_c = None
    try:
        respose_c = requests.get(data_href)
    except Exception as err:
        response_c = None
    
    if response_c != None:
        soupc = BeautifulSoup(response_c.text, "lxml")
        links_c = soupc.find_all("a")
        for l in links_c:
            row = [l.get("data-label"),l.text.replace("\n",""),l.get("href")]
            data.append(row)

df = pd.DataFrame(data,columns=['label','text','href'])
df.to_sql('DETIK', conn, if_exists='replace', index = False)
